<a href="https://colab.research.google.com/github/AppStrom/firebase-amazon-import/blob/main/parse_and_clean_meta_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen


In [12]:
import firebase_admin
from firebase_admin import credentials, firestore
cred = credentials.Certificate("service_account.json")
firebase_admin.initialize_app(cred)

firestore_db = firestore.client()

In [16]:
import time, sys
from IPython.display import clear_output

def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/All_Amazon_Meta.json.gz

--2021-08-03 19:47:06--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/All_Amazon_Meta.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12327618560 (11G) [application/octet-stream]
Saving to: ‘All_Amazon_Meta.json.gz.1’

All_Amazon_Meta.jso 100%[===================>]  11.48G  87.0MB/s    in 2m 19s  

2021-08-03 19:49:25 (84.4 MB/s) - ‘All_Amazon_Meta.json.gz.1’ saved [12327618560/12327618560]



In [ ]:
### load the meta data

data = []
start = 101
end = 100000
count = start

number_of_elements = 15023059

with gzip.open('All_Amazon_Meta.json.gz') as f: #15,023,059 records
    for l in f:
        count+=1
        #data.append(json.loads(l.strip()))
        update_progress(count / end)
        if(count>=start and count<=end):
          firestore_db.collection(u'amazon-products').add(json.loads(l.strip()))
          continue
        elif (count<start):
          continue
        else:
          break;
    
# total length of list, this number equals total number of products
#print(len(data))

# first row of the list
#print(data[0])

print(count)

Progress: [##------------------] 11.0%


(seconds: 1628021701
 nanos: 510065000,
 <google.cloud.firestore_v1.document.DocumentReference at 0x7fe538e3b510>)

In [6]:
# convert list into pandas dataframe

df = pd.DataFrame.from_dict(data)

print(len(df))

1000001


In [7]:
### remove rows with unformatted title (i.e. some 'title' may still contain html style content)

df3 = df.fillna('')
df4 = df3[df3.title.str.contains('getTime')] # unformatted rows
df5 = df3[~df3.title.str.contains('getTime')] # filter those unformatted rows
print(len(df4))
print(len(df5))

0
1000001


In [8]:
# how those unformatted rows look like
df4.iloc[0]

IndexError: ignored